In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
df = pd.read_parquet(f"{Path.home()}/data_ROHO/blooms-ml-stations.parquet")

In [ ]:
df

In [ ]:
df_station = df[df['station'] == 7][df['s_rho'] == -0.06]
df_station.set_index('ocean_time', inplace=True)
df_station

In [ ]:
df_station['P1_netPI'].plot(figsize=(14, 7))

In [ ]:
df_station['P1_c'].plot(figsize=(14, 7))

In [ ]:
df_station['light_PAR0'].plot(figsize=(14, 7))

In [ ]:
df_station['w'].plot(figsize=(14, 7))

In [ ]:
df_station['N1_p'].plot(figsize=(14, 7))